<a href="https://colab.research.google.com/github/CaritoRamos/Admission-Exam-Results-Analysis-II/blob/main/Web_Scraping_Admission_Exam_Results_Analysis_II_(UNMSM_2025_II).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalacion del paquete de Selenium
!pip install selenium

In [ ]:
#Importando librerías
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from google.colab import files
from urllib.parse import urljoin
import time
import json
import csv
import pandas as pd
import numpy as np

In [ ]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,540 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,126 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd

In [ ]:
#1. Configuración Inicial del WebDriver
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Ejecutar en modo sin cabeza
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

In [ ]:
#2. Función para esperar a cargar elementos en la página
def wait_for_element(driver, by, value, timeout=10):  #Espera 10 segundos a que el elemento aparezca
    try:
        element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((by, value)))
        return element
    except TimeoutException:
        print(f"Elemento no encontrado: {value}")
        return None

In [ ]:
#3. Función para Extraer Datos de una Carrera
def extract_results_data(driver, area, modalidad, carrera):
    print(f"\nExtrayendo datos de: {carrera}")

    #Busca la tabla de postulantes
    tabla = wait_for_element(driver, By.ID, 'tablaPostulantes')
    if not tabla:
        return pd.DataFrame()

    #Extrae los datos de la tabla
    datos = []
    filas = tabla.find_elements(By.TAG_NAME, 'tr')[1:]  #Se salta el encabezado

    for fila in filas:  #Recorre cada fila y extrae el código, nombres, escuela, etc.
        celdas = fila.find_elements(By.TAG_NAME, 'td')
        if len(celdas) >= 6:  # Asegurarse de que es una fila de datos
            codigo = celdas[0].text.strip()
            nombres = celdas[1].text.strip()
            escuela = celdas[2].text.strip()
            puntaje = celdas[3].text.strip()
            merito = celdas[4].text.strip()
            observacion = celdas[5].text.strip()

            datos.append({
                'CODIGO': codigo,
                'APELLIDOS_Y_NOMBRES': nombres,
                'ESCUELA_PROFESIONAL': escuela,
                'PUNTAJE': puntaje,
                'MERITO_EP': merito,
                'OBSERVACION': observacion,
                'AREA': area,
                'MODALIDAD': modalidad
            })

    return pd.DataFrame(datos)   #Devuelve un dataframe con la información extraída

In [ ]:
#4. Función para manejar la paginación
def process_pagination(driver, area, modalidad, carrera):
    all_data = pd.DataFrame()
    page_num = 1

    while True:  #Recorre todas las páginas de resultados.
        print(f"Procesando página {page_num}...")

        #Extrae datos de la página actual
        page_data = extract_results_data(driver, area, modalidad, carrera)
        all_data = pd.concat([all_data, page_data], ignore_index=True)

        #Pasa a la siguiente página
        try:
            #Espera a que el botón esté presente y sea clickeable
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#tablaPostulantes_next a')))

            if 'disabled' in next_button.get_attribute('class'): #Verifica si el botón "Siguiente" está habilitado.
                print("No hay más páginas")
                break   #Si no hay más páginas, sale del bucle.

            #Se desplaza al elemento antes de hacer clic
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(1)

            #Hace clic con JavaScript como alternativa
            try:
                next_button.click()
            except:
                driver.execute_script("arguments[0].click();", next_button)

            #Espera a que la nueva página cargue
            WebDriverWait(driver, 10).until(
                EC.staleness_of(next_button))

            page_num += 1
            time.sleep(2)

        except TimeoutException:
            print("Tiempo de espera agotado para el botón siguiente")
            break
        except Exception as e:
            print(f"Error al cambiar de página: {str(e)}")
            break

    return all_data

In [ ]:
#5. Función para procesar las carreras de una modalidad
def process_carreras(driver, base_url, area, modalidad, modalidad_url):
    print(f"\nProcesando modalidad: {modalidad}")

    #Navegando a la página de la modalidad
    driver.get(modalidad_url)
    time.sleep(2)

    #Busca la tabla de carreras
    tabla_carreras = wait_for_element(driver, By.CSS_SELECTOR, 'table.table-bordered')
    if not tabla_carreras:
        return pd.DataFrame()

    #Extrae enlaces de carreras
    enlaces_carreras = []
    filas = tabla_carreras.find_elements(By.TAG_NAME, 'tr')[1:]  #Se salta el encabezado

    for fila in filas:  #Recorre todas las carreras de una modalidad.
        enlace = fila.find_element(By.TAG_NAME, 'a')
        nombre_carrera = enlace.text.strip()
        url_carrera = enlace.get_attribute('href')
        enlaces_carreras.append((nombre_carrera, url_carrera))

    print(f"Encontradas {len(enlaces_carreras)} carreras para esta modalidad")

    #Procesa cada carrera
    all_data = pd.DataFrame()

    for nombre_carrera, url_carrera in enlaces_carreras:
        driver.get(url_carrera)
        time.sleep(2)

        #Configuración para mostrar todos los registros de 100 en 100 (en el selector de la página)
        try:
            select = driver.find_element(By.NAME, 'tablaPostulantes_length')
            options = select.find_elements(By.TAG_NAME, 'option')
            for option in options:
                if option.get_attribute('value') == '100':
                    option.click()
                    time.sleep(2)
                    break
        except NoSuchElementException:
            pass

        #Procesa la carrera con paginación
        carrera_data = process_pagination(driver, area, modalidad, nombre_carrera)
        all_data = pd.concat([all_data, carrera_data], ignore_index=True)

    return all_data

In [ ]:
#6. Función para procesar una área completa
def process_area(driver, base_url, area, area_url):
    print(f"\nProcesando área: {area}")

    #Navega a la página del área
    driver.get(area_url)
    time.sleep(2)

    #Encuentra la tabla de modalidades
    tabla_modalidades = wait_for_element(driver, By.CSS_SELECTOR, 'table.table-bordered')
    if not tabla_modalidades:
        return pd.DataFrame()

    #Extrae todos los enlaces de modalidades
    enlaces_modalidades = []
    filas = tabla_modalidades.find_elements(By.TAG_NAME, 'tr')[1:]  # Saltar encabezado

    for fila in filas:
        enlace = fila.find_element(By.TAG_NAME, 'a')
        nombre_modalidad = enlace.text.strip()
        url_modalidad = enlace.get_attribute('href')
        enlaces_modalidades.append((nombre_modalidad, url_modalidad))

    print(f"Encontradas {len(enlaces_modalidades)} modalidades para esta área")

    #Procesa cada modalidad
    all_data = pd.DataFrame()

    for nombre_modalidad, url_modalidad in enlaces_modalidades:
        modalidad_data = process_carreras(driver, base_url, area, nombre_modalidad, url_modalidad)
        all_data = pd.concat([all_data, modalidad_data], ignore_index=True)

    return all_data

In [ ]:
#7. Función principal
def main():
    #URLs de las áreas
    areas = {
        "A": "https://admision.unmsm.edu.pe/Website20252GeneralA/",
        "B,C,D y E": "https://admision.unmsm.edu.pe/Website20252General/",
        "ESPECIAL": "https://admision.unmsm.edu.pe/Website20252Especial/",
        "DEP. CALIF.": "https://admision.unmsm.edu.pe/Website20252Decan/"
    }

    #Configura el driver
    driver = setup_driver()

    try:
        #DataFrame para almacenar todos los resultados
        resultados_finales = pd.DataFrame()

        #Procesa cada área
        for area_nombre, area_url in areas.items():
            area_data = process_area(driver, area_url, area_nombre, area_url)
            resultados_finales = pd.concat([resultados_finales, area_data], ignore_index=True)

            #Guardar resultados parciales
            resultados_finales.to_csv('resultados_admision_unmsm.csv', index=False, encoding='utf-8-sig')
            print(f"\nDatos guardados temporalmente. Total de registros: {len(resultados_finales)}")

        #Guarda resultados finales
        resultados_finales.to_csv('resultados_admision_unmsm_final.csv', index=False, encoding='utf-8-sig')
        print("\nProceso completado. Resultados guardados en 'resultados_admision_unmsm_final.csv'")

    except Exception as e:
        print(f"\nError durante la ejecución: {str(e)}")
    finally:
        driver.quit()

In [ ]:
#8. Ejecutando la función principal
if __name__ == "__main__":
    main()


Procesando área: A
Encontradas 6 modalidades para esta área

Procesando modalidad: EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁSICA ALTERNATIVA (EBA)
Encontradas 18 carreras para esta modalidad
Procesando página 1...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 2...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 3...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 4...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 5...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 6...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 7...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 8...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 9...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 10...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 11...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 12...

Extrayendo datos de: MEDICINA HUMANA
Procesando página 13...

Extrayendo datos de: MEDICINA HUMANA
Procesa

In [ ]:
#9. Descargando el archivo generado
files.download('resultados_admision_unmsm_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>